Cryptocurrency trading bot.

Global variable definitions.

In [1]:
intervals = ['1m']

api_key = 'yoyoyo'
api_secret = 'yeyeye'

Installs.

In [2]:
!pip install python-binance --upgrade --no-cache-dir
#!pip install pixiedust --upgrade --no-cache-dir

Requirement already up-to-date: python-binance in ./anaconda3/lib/python3.7/site-packages (0.7.5)


Library imports.

In [3]:
from binance.client import Client

#import pixiedust
#import pdb
import math
import random
import time
import datetime
import numpy as np
import pandas as pd

Function definitions.

In [4]:
class Cryptocurrency_indicators:
    def _calculate_moving_average_(self, dataset, window=2, method='simple'):
        dataset = dataset.astype(float)

        if method == 'simple':
            dataset = dataset.rolling(window=window)
        elif method == 'exponential':
            dataset = dataset.ewm(span=window)

        dataset = dataset.mean()
        dataset = dataset.fillna(method='backfill', axis='index')
        return dataset

    def calculate_simple_moving_average(self, dataset, window=2):
        return self._calculate_moving_average_(dataset, window=window, method='simple')

    def calculate_exponential_moving_average(self, dataset, window=2):
        return self._calculate_moving_average_(dataset, window=window, method='exponential')

    def _calculate_heikin_ashi_(self, df):
        df = df.tz_localize(tz=None, ambiguous='infer')
        df = df[['open', 'high', 'low', 'close']].copy().astype('float')
        heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['open', 'high', 'low', 'close'])
        heikin_ashi_df['close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 6

        for i in range(len(df)):
            if i == 0:
                heikin_ashi_df.iat[0, 0] = df['open'].iloc[0]
            else:
                heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i - 1, 0] + heikin_ashi_df.iat[i - 1, 3]) / 2

        heikin_ashi_df['high'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['high']).max(axis=1)
        heikin_ashi_df['low'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['low']).min(axis=1)

        return heikin_ashi_df

    def calculate_heikin_ashi(self, df):
        heikin_ashi = self.calculate_simple_moving_average(df, window=1)
        heikin_ashi = self._calculate_heikin_ashi_(heikin_ashi)
        heikin_ashi = self.calculate_exponential_moving_average(heikin_ashi, window=1)

        return heikin_ashi

    def _calculate_RSI_(self, ticker, span=14):
        up = ticker['close'].astype(float).pct_change()
        down = up.copy()
        up[up < 0.0] = 0.0
        down[down > 0.0] = 0.0

        up = up.ewm(span=span).mean().fillna(method='pad')
        down = down.ewm(span=span).mean().abs().fillna(method='pad')

        ticker = up / down
        ticker = 100 - (100 / (1 + ticker))

        ticker = ticker.replace([-np.inf, np.inf], np.nan)
        ticker.iloc[0] = ticker.iloc[1]
        return ticker.fillna(method='pad')

    def calculate_RSI_6(self, ticker):
        return self._calculate_RSI_(ticker, span=6)

    def calculate_RSI_12(self, ticker):
        return self._calculate_RSI_(ticker, span=12)

    def calculate_MACD(self, ticker, span1=12, span2=26, average='exponential'):
        shorter = self._calculate_moving_average_(ticker, window=span1, method=average)['close']
        longer = self._calculate_moving_average_(ticker, window=span2, method=average)['close']

        ticker = (shorter - longer).replace([-np.inf, np.inf], np.nan)
        ticker.iloc[0] = ticker.iloc[1]
        ticker = ticker.fillna(method='pad').to_frame(name='MACD')
        ticker['signal_line'] = ticker.MACD.ewm(span=9).mean()
        ticker['histogram'] = ticker.MACD - ticker.signal_line
        return ticker

    def calculate_ATR(self, ticker, min_periods=14):
        ticker = ticker.astype(float)
        up = pd.DataFrame([ticker['high'].shift(), 
                           ticker['close']]).fillna(method='backfill', axis='columns').max(axis='index')

        down = pd.DataFrame([ticker['low'].shift(), 
                             ticker['close']]).fillna(method='backfill', axis='columns').min(axis='index')

        ticker = (up - down).ewm(alpha=1 / min_periods, 
                                 min_periods=min_periods, 
                                 adjust=False).mean()

        ticker = ticker.replace([-np.inf, np.inf], np.nan)
        return ticker.fillna(method='backfill')

    def calculate_bollinger_bands(self, dataset, period=20):
        df = dataset.astype(float).copy()
        df['MA'] = self._calculate_moving_average_(df, window=period, method='simple')['close']
        df['BB_up'] = df['MA'] + df['MA'].rolling(period).std()
        df['BB_down'] = df['MA'] - df['MA'].rolling(period).std()
        df['BB_width'] = df['BB_up'] - df['BB_down']
        return df[['BB_up', 'BB_down', 'BB_width']].dropna()

    def calculate_average_directional_index(self, df, n=14, n_ADX=14):
        df = df.astype(float)
        dataset_index = df.index.copy()
        df = df.reset_index().drop(columns=['time']).astype(float)
        i = 0
        UpI = []
        DoI = []

        while i + 1 <= df.index[-1]:
            UpMove = df.loc[i + 1, 'high'] - df.loc[i, 'high']
            DoMove = df.loc[i, 'low'] - df.loc[i + 1, 'low']

            if UpMove > DoMove and UpMove > 0:
                UpD = UpMove
            else:
                UpD = 0

            UpI.append(UpD)

            if DoMove > UpMove and DoMove > 0:
                DoD = DoMove
            else:
                DoD = 0

            DoI.append(DoD)
            i += 1

        ATR = self.calculate_ATR(df, min_periods=14)
        UpI = pd.Series(UpI)
        DoI = pd.Series(DoI)
        PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean() / ATR, name='PosDI')
        NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean() / ATR, name='NegDI')
        ADX = pd.Series((abs(PosDI - NegDI) / (PosDI + NegDI)).ewm(span=n_ADX, 
                                                                   min_periods=n_ADX).mean(), 
                        name='ADX')

        df = df.join(ADX).join(PosDI).join(NegDI)
        df.index = dataset_index
        return df[['ADX', 'PosDI', 'NegDI']].dropna()

    def calculate_commodity_channel_index(self, dataset, min_periods=20):
        dataset = dataset.astype(float)
        PP = (dataset['high'] + dataset['low'] + dataset['close']) / 3
        CCI = pd.Series((PP - PP.rolling(min_periods, min_periods=min_periods).mean()) / \
                        PP.rolling(min_periods, min_periods=min_periods).std(),
                        name='CCI')
        return dataset.join(CCI)['CCI']

    def calculate_KDJ(self, dataset):
        def get_rsv(dataset):
            low_min = dataset['low'].rolling(min_periods=1, window=9, center=False).min()
            high_max = dataset['high'].rolling(min_periods=1, window=9, center=False).max()
            return ((dataset['close'] - low_min) / (high_max - low_min)).fillna(0).astype(float) * 100

        def calc_kd(column):
            k = 50.0
            for i in (1.0 / 3.0) * column:
                k = (2.0 / 3.0) * k + i
                yield k

        dataset = dataset.astype(float)
        dataset['K'] = list(calc_kd(get_rsv(dataset)))
        dataset['D'] = list(calc_kd(dataset['K']))
        dataset['J'] = 3 * dataset['K'] - 2 * dataset['D']
        return dataset[['K', 'D', 'J']]

    def calculate_relative_volume_level(self, 
                                        dataset, 
                                        average1=26, 
                                        average2=14, 
                                        method='simple'):

        dataset = dataset.astype(float)

        volume_average = self._calculate_moving_average_(dataset, window=average1, method=method)

        relative_volume = dataset / average1

        smoothed_relative_volume = self._calculate_moving_average_(relative_volume, window=average2, method=method)

        return smoothed_relative_volume[['volume']].pct_change()


class Cryptocurrency_triggers(Cryptocurrency_indicators):
    def update(self, dataset):
        self.momentum_trigger = self.calculate_momentum_trigger(dataset)
        self.volatility_trigger = self.calculate_volatility_trigger(dataset)
        self.MACD_trigger = self.calculate_MACD_trigger(dataset)
        self.trend_strength_trigger = self.calculate_trend_strength_trigger(dataset)
        self.real_trigger = self.calculate_real_trigger(dataset)
        self.stop_loss_trigger = self.calculate_stop_loss_trigger(dataset)

    def calculate_trend_trigger(self, dataset):
        heikin_ashi = self.calculate_heikin_ashi(dataset)
        return (heikin_ashi['close'] - heikin_ashi['open']) > 0

    def calculate_overtraded_trigger(self, dataset):
        RSI_6 = self.calculate_RSI_6(dataset)
        RSI_12 = self.calculate_RSI_12(dataset)
        return RSI_6 > RSI_12

    def calculate_trend_strength_trigger(self, dataset):
        ADX = self.calculate_average_directional_index(dataset)
        return ADX['ADX'] > 0.25

    def calculate_trend_strength_positive_trigger(self, dataset):
        ADX = self.calculate_average_directional_index(dataset)
        return ADX['PosDI'] > ADX['NegDI']

    def calculate_trend_strength_negative_trigger(self, dataset):
        ADX = self.calculate_average_directional_index(dataset)
        return ADX['PosDI'] < ADX['NegDI']

    def calculate_momentum_trigger(self, dataset):
        KDJ = self.calculate_KDJ(dataset)
        return KDJ['J'] > KDJ['D']

    def calculate_volatility_trigger(self, dataset):
        dataset = dataset.astype(float)
        bollinger_bands = self.calculate_bollinger_bands(dataset)
        return (bollinger_bands['BB_width'] / dataset['close']) > 0.0005

    def calculate_MACD_trigger(self, dataset):
        dataset = dataset.astype(float)
        MACD = self.calculate_MACD(dataset)
        return MACD['histogram'] > 0

    def calculate_stop_loss_trigger(self, dataset):
        dataset = dataset.astype(float)
        stop_loss = dataset - self.calculate_ATR(dataset)
        return dataset['open'] < stop_loss['close']

    def calculate_real_trigger(self, dataset):
        dataset = dataset.astype(float)
        return (dataset['close'] - dataset['open']) > 0

    def calculate_relative_volume_level_trigger(self, dataset, threshold=1.75):
        relative_volume_level = self.calculate_relative_volume_level(dataset)
        return relative_volume_level['volume'] > threshold


class Cryptocurrency_pair_info:
    def __init__(self, client, pair):
        self.client = client
        self.pair = pair

        pair_info = client.get_symbol_info(self.pair)
        self.base_asset = pair_info['baseAsset']
        self.quote_asset = pair_info['quoteAsset']
        self.precision = pair_info['quotePrecision']
        self.base_asset_precision = pair_info['baseAssetPrecision']

        filters = pair_info['filters']
        price_filter = [ticker for ticker in filters if ticker['filterType'] == 'PRICE_FILTER']
        lot_size = [ticker for ticker in filters if ticker['filterType'] == 'LOT_SIZE']
        self.tick_size = [ticker['tickSize'].find('1') - 2 for ticker in price_filter][0]
        self.step_size = [ticker['stepSize'].find('1') - 2 for ticker in lot_size][0]

        self.calculate_balance()

    def calculate_balance(self):
        self.base_asset_balance = float(self.client.get_asset_balance(asset=self.base_asset)['free'])        
        self.quote_asset_balance = float(self.client.get_asset_balance(asset=self.quote_asset)['free'])
        self.pair_last_price = float(self.client.get_ticker(symbol=self.pair)['lastPrice'])
        self.pair_buy_balance = float(self.quote_asset_balance) / float(self.pair_last_price)
        self.pair_sell_balance = float(self.base_asset_balance) * float(self.pair_last_price)
        self.pair_combined_base_balance = float(self.pair_buy_balance) + float(self.base_asset_balance)
        self.pair_combined_quote_balance = float(self.pair_sell_balance) + float(self.quote_asset_balance)

        self.base_asset_balance = "{:0.0{}f}".format(float(self.base_asset_balance), self.base_asset_precision).rstrip('0').rstrip('.')
        self.quote_asset_balance = "{:0.0{}f}".format(float(self.quote_asset_balance), self.precision).rstrip('0').rstrip('.')
        self.pair_last_price = "{:0.0{}f}".format(float(self.pair_last_price), self.precision).rstrip('0').rstrip('.')
        self.pair_buy_balance = "{:0.0{}f}".format(float(self.pair_buy_balance), self.base_asset_precision).rstrip('0').rstrip('.')
        self.pair_sell_balance = "{:0.0{}f}".format(float(self.pair_sell_balance), self.precision).rstrip('0').rstrip('.')
        self.pair_combined_base_balance = "{:0.0{}f}".format(float(self.pair_combined_base_balance), self.base_asset_precision).rstrip('0').rstrip('.')
        self.pair_combined_quote_balance = "{:0.0{}f}".format(float(self.pair_combined_quote_balance), self.precision).rstrip('0').rstrip('.')

    def print_balance(self):
        print("\n")
        print('pair: ', self.pair)
        print('base_asset_balance: ', self.base_asset_balance)
        print('quote_asset_balance: ', self.quote_asset_balance)
        print('pair_last_price: ', self.pair_last_price)
        print('pair_buy_balance: ', self.pair_buy_balance)
        print('pair_sell_balance: ', self.pair_sell_balance)
        print('pair_combined_base_balance: ', self.pair_combined_base_balance)
        print('pair_combined_quote_balance: ', self.pair_combined_quote_balance)
        print("\n")


class Cryptocurrency_pair_at_interval:
    def __init__(self, client, info, interval, download=False):
        self.dataset = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])
        self.interval = interval
        self.period = self.get_n_periods_from_time(n=60)
        self.indicators = Cryptocurrency_triggers()

        if download:
            self.dataset = self.download_dataset(client=client, pair=info.pair)
            self.indicators.update(self.dataset)

    def get_n_periods_from_time(self, n=60):
        return str(int(self.interval[:-1]) * n) + self.interval[-1:]

    def download_dataset(self, client, pair):
        dataset = \
            client.get_historical_klines(symbol=pair, 
                                         interval=self.interval, 
                                         start_str=self.period)

        dataset = pd.DataFrame(dataset, 
                               columns=['time', 
                                        'open', 
                                        'high', 
                                        'low', 
                                        'close', 
                                        'volume', 
                                        'Close time', 
                                        'Quote asset volume', 
                                        'Number of trades', 
                                        'Taker buy base asset volume', 
                                        'Taker buy quote asset volume', 
                                        'Ignore'])

        four_hours = 14400
        milliseconds = 1000

        dataset['time'] = \
            dataset['time'].apply(lambda timestamp: \
                                  datetime.datetime.fromtimestamp((timestamp / \
                                                                   milliseconds) - \
                                                                  four_hours))

        dataset = dataset[['time', 'open', 'high', 'low', 'close', 'volume']]
        dataset.set_index('time', inplace=True)
        return dataset.applymap(lambda entry: entry.rstrip('0').rstrip('.'))


class Cryptocurrency_trader:
    def __init__(self, client, pair):
        self.state = 'exit'
        self.is_tradable = True
        self.info = Cryptocurrency_pair_info(client=client, pair=pair)
        self.calculate_position()

    def calculate_position(self):
        if float(self.info.pair_buy_balance) > float(self.info.base_asset_balance):
            self.position = 'sell'
        elif float(self.info.base_asset_balance) > float(self.info.pair_buy_balance):
            self.position = 'buy'

    def trade_pair(self, percentage_to_trade=0.999999):
        self.calculate_position()

        if self.position == 'sell':
            coins_available = float(self.info.pair_buy_balance)
            side = Client.SIDE_BUY
            position = 'buy'
        elif self.position == 'buy':
            coins_available = float(self.info.base_asset_balance)
            side = Client.SIDE_SELL
            position = 'sell'

        coins_available *= percentage_to_trade
        quantity = math.floor(coins_available * 10**self.info.step_size) / \
                        float(10**self.info.step_size)

        if self.info.tick_size < 0:
            quantity = math.floor(coins_available * abs(self.info.tick_size)) / \
                            float(abs(self.info.tick_size))

        quantity = "{:0.0{}f}".format(float(quantity), self.info.precision).rstrip('0').rstrip('.')

        print('traded quantity:', quantity)
        self.info.calculate_balance()
        client.create_order(symbol=self.info.pair, 
                            side=side, 
                            type=Client.ORDER_TYPE_MARKET, 
                            quantity=quantity, 
                            recvWindow=2000)

        self.info.calculate_balance()
        self.info.print_balance()

        if self.state == 'exit':
            self.state = 'entry'
        elif self.state == 'entry':
            self.state = 'exit'

        print("\nPosition for base asset " + self.info.base_asset + ' is ' + position + '.')
        print('Position for quote asset ' + self.info.quote_asset + ' is ' + self.position + '.')
        print('State is ' + self.state + ".\n")
        self.position = position


class Cryptocurrency_pair(Cryptocurrency_trader):
    def __init__(self, 
                 client, 
                 pair, 
                 intervals=intervals, 
                 download=False):

        self.intervals = intervals
        self.download = download
        super().__init__(client, pair)
        self.interval = self.get_datasets(client, intervals)

    def get_datasets(self, client, intervals):
        dataset = dict()

        for interval in intervals:
            dataset[interval] = Cryptocurrency_pair_at_interval(client=client, 
                                                                info=self.info, 
                                                                interval=interval, 
                                                                download=self.download)

        return dataset

    def trade(self, interval=intervals[0]):
        pair_at_interval = self.interval[interval]
        pair_at_interval.indicators.update(pair_at_interval.dataset)
        self.info.calculate_balance()
        self.info.print_balance()

        volatility_trigger = pair_at_interval.indicators.volatility_trigger.iloc[-1]
        MACD_trigger = pair_at_interval.indicators.MACD_trigger.iloc[-1]
        real_trigger = pair_at_interval.indicators.real_trigger.iloc[-1]
        momentum_trigger = pair_at_interval.indicators.momentum_trigger.iloc[-1]
        trend_strength_trigger = pair_at_interval.indicators.trend_strength_trigger.iloc[-1]
        stop_loss_trigger = pair_at_interval.indicators.stop_loss_trigger.iloc[-1]

        if self.state == 'exit':
            if self.position == 'sell' and \
                    volatility_trigger and \
                    MACD_trigger and \
                    real_trigger and \
                    momentum_trigger and \
                    trend_strength_trigger:
                self.trade_pair()
            elif self.position == 'buy' and \
                    volatility_trigger and \
                    not MACD_trigger and \
                    not real_trigger and \
                    not momentum_trigger and \
                    trend_strength_trigger:
                self.trade_pair()
        elif self.state == 'entry':
            if self.position == 'sell' and not real_trigger:
                self.trade_pair()
            elif self.position == 'buy' and not real_trigger:
                self.trade_pair()

In [ ]:
quote_asset = 'BTC'
client = Client(api_key, api_secret)

def get_tickers():
    tickers = pd.DataFrame(client.get_ticker())[['symbol', 'priceChangePercent']]
    tickers['priceChangePercent'] = tickers['priceChangePercent'].astype(float)
    return tickers[tickers['symbol'].str.endswith(quote_asset)]

def get_rising_pairs(tickers, threshold):
    old_tickers = tickers.copy()
    tickers = get_tickers()
    difference = tickers['priceChangePercent'] - old_tickers['priceChangePercent']
    pairs = tickers['symbol'][threshold(difference)].tolist()
    return tickers, pairs

tickers = get_tickers()

while True:
    time.sleep(5)

    tickers, rising_pairs = \
        get_rising_pairs(tickers, threshold=lambda difference: difference > 0.5)

    for symbol in rising_pairs:
        pair = Cryptocurrency_pair(client, symbol, intervals=intervals, download=True)

        if pair.info.pair_combined_base_balance != 0:
            try:
                pair.trade()
            except:
                pass

            while pair.state == 'entry':
                time.sleep(5)

                tickers, sinking_pairs = \
                    get_rising_pairs(tickers, threshold=lambda difference: difference < 0.0)

                if pair.info.pair in sinking_pairs:
                    try:
                        pair.trade_pair()
                        pair.state = 'exit'
                    except:
                        pass



pair:  OAXBTC
base_asset_balance:  0
quote_asset_balance:  0.00558543
pair_last_price:  0.00000605
pair_buy_balance:  923.21157025
pair_sell_balance:  0
pair_combined_base_balance:  923.21157025
pair_combined_quote_balance:  0.00558543




pair:  DOCKBTC
base_asset_balance:  0
quote_asset_balance:  0.00558543
pair_last_price:  0.00000072
pair_buy_balance:  7757.54166667
pair_sell_balance:  0
pair_combined_base_balance:  7757.54166667
pair_combined_quote_balance:  0.00558543




pair:  VIBEBTC
base_asset_balance:  0
quote_asset_balance:  0.00558543
pair_last_price:  0.00000163
pair_buy_balance:  3426.64417178
pair_sell_balance:  0
pair_combined_base_balance:  3426.64417178
pair_combined_quote_balance:  0.00558543




pair:  ZILBTC
base_asset_balance:  0
quote_asset_balance:  0.00558543
pair_last_price:  0.00000188
pair_buy_balance:  2970.97340426
pair_sell_balance:  0
pair_combined_base_balance:  2970.97340426
pair_combined_quote_balance:  0.00558543




pair:  GOBTC
base_asset_balanc